In [ ]:
 !pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.tajhotels.com/en-in/hotels/taj-palace-new-delhi/restaurants/capital-kitchen"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the specific div using aria-label
food_facilities_div = soup.find('div', {'aria-label': 'hotel-dine-in-food-facilities'})

if food_facilities_div:
    print(food_facilities_div.get_text(separator='\n', strip=True))
else:
    print("Div not found or content is loaded dynamically.")


BREAKFAST
6:30 am to 10:30 am
LUNCH
12:30 pm to 2:45 pm
DINNER
7:00 pm to 11:00 pm
CUISINE
Multi-Cuisine
DRESS CODE
Smart Casual
Menu
DOWNLOAD
CONTACT
2 Sardar Patel Marg Diplomatic Enclave, New Delhi, 110021, India
tphcapitalkitchen.del@tajhotels.com​
+91 11665 03721


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

url = "https://www.tajhotels.com/en-in/hotels/taj-palace-new-delhi/restaurants/capital-kitchen"
headers = {
    'User-Agent': 'Mozilla/5.0'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all links ending with .pdf
pdf_links = []
for a in soup.find_all('a', href=True):
    if a['href'].endswith('.pdf'):
        link = a['href']
        if not link.startswith('http'):
            link = "https://www.ihcltata.com" + link
        pdf_links.append(link)

# Fallback: Use regex to find any PDF links in the HTML
if not pdf_links:
    matches = re.findall(r'href=[\'"]?([^\'" >]+\.pdf)', response.text)
    for link in matches:
        if not link.startswith('http'):
            link = "https://www.ihcltata.com" + link
        pdf_links.append(link)

print("Menu PDF Links found:")
for link in pdf_links:
    print(link)


Menu PDF Links found:


In [ ]:
# Install required libraries
!pip install requests beautifulsoup4 PyPDF2 pandas

# Import necessary libraries
import requests
from bs4 import BeautifulSoup
import PyPDF2
import io
import re
import pandas as pd
import json

# 1. Function to scrape restaurant details from the website
def scrape_restaurant_details():
    url = "https://www.tajhotels.com/en-in/hotels/taj-palace-new-delhi/restaurants/capital-kitchen"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract restaurant name
        name = "Capital Kitchen"
        name_tag = soup.find('h1')
        if name_tag:
            name = name_tag.text.strip()

        # Extract location
        location = "2 Sardar Patel Marg Diplomatic Enclave, New Delhi, 110021, India"
        location_div = soup.find(string=re.compile("CONTACT", re.IGNORECASE))
        if location_div:
            location_p = location_div.find_next('p')
            if location_p:
                location = location_p.text.strip()

        # Extract contact information
        contact = {
            "phone": "+91 11665 03721",
            "email": "tphcapitalkitchen.del@tajhotels.com"
        }

        email_a = soup.find('a', href=lambda href: href and 'mailto:' in href)
        if email_a:
            contact["email"] = email_a.text.strip()

        phone_a = soup.find('a', href=lambda href: href and 'tel:' in href)
        if phone_a:
            contact["phone"] = phone_a.text.strip()

        # Extract timings
        timings = {
            "breakfast": "6:30 am to 10:30 am",
            "lunch": "12:30 pm to 2:45 pm",
            "dinner": "7:00 pm to 11:00 pm"
        }

        breakfast = soup.find(string=re.compile("BREAKFAST", re.IGNORECASE))
        if breakfast:
            breakfast_p = breakfast.find_next('p')
            if breakfast_p:
                timings["breakfast"] = breakfast_p.text.strip()

        lunch = soup.find(string=re.compile("LUNCH", re.IGNORECASE))
        if lunch:
            lunch_p = lunch.find_next('p')
            if lunch_p:
                timings["lunch"] = lunch_p.text.strip()

        dinner = soup.find(string=re.compile("DINNER", re.IGNORECASE))
        if dinner:
            dinner_p = dinner.find_next('p')
            if dinner_p:
                timings["dinner"] = dinner_p.text.strip()

    except Exception as e:
        print(f"Error scraping website: {e}")
        print("Using default values...")

    # Use the direct menu URL from search results
    menu_url = "https://www.ihcltata.com/content/dam/luxury/hotels/taj-palace-delhi/documents/Capital-Kitchen.pdf"

    return {
        "name": name,
        "location": location,
        "contact": contact,
        "timings": timings,
        "menu_url": menu_url
    }

# 2. Function to download and extract menu information from PDF
def extract_menu_from_pdf(pdf_url):
    print(f"Downloading PDF from: {pdf_url}")

    try:
        response = requests.get(pdf_url)

        if response.status_code == 200:
            # Save the PDF locally
            with open("menu.pdf", "wb") as f:
                f.write(response.content)

            print("PDF downloaded successfully. Extracting text...")

            # Extract text using PyPDF2
            pdf_file = io.BytesIO(response.content)
            pdf_reader = PyPDF2.PdfReader(pdf_file)

            # Extract text from all pages
            menu_text = ""
            for page in pdf_reader.pages:
                menu_text += page.extract_text() + "\n\n"

            # Process the text to extract menu items
            menu_items = []
            current_section = "Menu Items"
            lines = menu_text.split('\n')

            for i, line in enumerate(lines):
                line = line.strip()
                if not line:
                    continue

                # Check if this line could be a section header (all caps, no price)
                if line.isupper() and not re.search(r'\d{3,4}$', line):
                    current_section = line
                    continue

                # Look for menu items with prices (item followed by 3-4 digit number)
                price_match = re.search(r'(\d{3,4})$', line)
                if price_match:
                    price = price_match.group(1)
                    # Extract the item name (everything before the price)
                    item_name = line[:line.rfind(price)].strip()

                    # Look for description in the next line
                    description = ""
                    if i + 1 < len(lines):
                        next_line = lines[i + 1].strip()
                        # If next line doesn't have a price and isn't a section header, it's likely a description
                        if not re.search(r'\d{3,4}$', next_line) and not next_line.isupper() and next_line:
                            description = next_line

                    menu_items.append({
                        "section": current_section,
                        "item": item_name,
                        "price": "₹" + price,
                        "description": description
                    })

            return menu_items
        else:
            print(f"Failed to download PDF. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error extracting menu: {e}")
        return None

# Main function to run the scraping and display results
def scrape_capital_kitchen():
    print("==== Capital Kitchen Restaurant Scraper ====")

    # Get restaurant details
    restaurant_details = scrape_restaurant_details()

    # Display restaurant details
    print("\n=== Restaurant Details ===")
    print(f"Name: {restaurant_details['name']}")
    print(f"Location: {restaurant_details['location']}")
    print(f"Contact: {restaurant_details['contact']['phone']} | {restaurant_details['contact']['email']}")
    print("\nTimings:")
    print(f"  Breakfast: {restaurant_details['timings']['breakfast']}")
    print(f"  Lunch: {restaurant_details['timings']['lunch']}")
    print(f"  Dinner: {restaurant_details['timings']['dinner']}")
    print(f"\nMenu URL: {restaurant_details['menu_url']}")

    # Extract menu from PDF
    print("\nExtracting menu from PDF...")
    menu_items = extract_menu_from_pdf(restaurant_details['menu_url'])

    if menu_items:
        # Create DataFrame to display menu items
        menu_df = pd.DataFrame(menu_items)

        # Save to CSV and JSON
        menu_df.to_csv("capital_kitchen_menu.csv", index=False)
        with open("capital_kitchen_menu.json", "w") as f:
            json.dump(menu_items, f, indent=2)

        # Display sample items
        print(f"\nExtracted {len(menu_items)} menu items. Sample items:")
        print(menu_df.head(10))
        print("\nData saved to 'capital_kitchen_menu.csv' and 'capital_kitchen_menu.json'")
    else:
        print("Failed to extract menu items.")

    # Save restaurant details
    with open("restaurant_details.json", "w") as f:
        json.dump(restaurant_details, f, indent=2)

    print("Restaurant details saved to 'restaurant_details.json'")

    return restaurant_details, menu_items

# Run the scraper
restaurant_details, menu_items = scrape_capital_kitchen()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.7 MB/s eta 0:00:00
==== Capital Kitchen Restaurant Scraper ====

=== Restaurant Details ===
Name: Capital Kitchen
Location: 2 Sardar Patel Marg Diplomatic Enclave, New Delhi, 110021, India
Contact: 1-800-111-825 | reservations@ihcltata.com

Timings:
  Breakfast: 6:30 am to 10:30 am
  Lunch: 12:30 pm to 2:45 pm
  Dinner: 7:00 pm to 11:00 pm

Menu URL: https://www.ihcltata.com/content/dam/luxury/hotels/taj-palace-delhi/documents/Capital-Kitchen.pdf

Extracting menu from PDF...
PDF downloaded successfully. Extracting text...

Extracted 100 menu items. Sample items:
      section                                               item  price  \
0  Menu Items  All prices are in Indian Rupees and subject to...  ₹1450   
1  Menu Items                                    PIZZA PEPPERONI  ₹1450   
2  Menu Items                           THE CAPITAL PIZZA 1450 /  ₹1150   
3  Menu Items                           CLASSIC PIZZA MARGHERITA  ₹115